##ThinkDSP

This notebook contains code solutions to exercises in Chapter 3: Non-periodic signals

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

import thinkdsp
import thinkplot

%matplotlib inline

**Exercise 3.2**: Write a class called `SawtoothChirp` that extends `Chirp` and overrides evaluate to generate a sawtooth waveform with frequency that increases (or decreases) linearly.

In [ ]:
import math
PI2 = 2 * math.pi

class SawtoothChirp(thinkdsp.Chirp):
    """Represents a sawtooth signal with varying frequency."""

    def _evaluate(self, ts, freqs):
        """Helper function that evaluates the signal.

        ts: float array of times
        freqs: float array of frequencies during each interval
        """
        dts = numpy.diff(ts)
        dps = PI2 * freqs * dts
        phases = numpy.cumsum(dps)
        phases = numpy.insert(phases, 0, 0)
        cycles = phases / PI2
        frac, _ = numpy.modf(cycles)
        ys = thinkdsp.normalize(thinkdsp.unbias(frac), self.amp)
        return ys


Here's what it sounds like.

In [ ]:
signal = SawtoothChirp(start=220, end=880)
wave = signal.make_wave(duration=2, framerate=10000)
wave.apodize()
wave.make_audio()

And here's the spectrogram.

In [ ]:
sp = wave.make_spectrogram(1024)
sp.plot()
thinkplot.config(xlabel='time (s)', ylabel='frequency (Hz)', legend=False)

At a relatively low frame rate, you can see the aliased harmonics bouncing off the folding frequency.  And you can hear them as a background hiss.  If you crank up the frame rate, they go away.

**Exercise 3.3**: In musical terminology, a “glissando” is a note that slides from one pitch to another, so it is similar to a chirp. A trombone player can play a glissando by extending the trombone slide while blowing continuously. As the slide extends, the total length of the tube gets longer, and the resulting pitch is inversely proportional to length.

In [ ]:
class TromboneGliss(thinkdsp.Chirp):
    """Represents a trombone-like signal with varying frequency."""
    
    def evaluate(self, ts):
        """Evaluates the signal at the given times.

        ts: float array of times
        
        returns: float wave array
        """
        l1, l2 = 1.0 / self.start, 1.0 / self.end
        lengths = numpy.linspace(l1, l2, len(ts)-1)
        freqs = 1 / lengths
        return self._evaluate(ts, freqs)

Write a function that simulates a trombone glissando from C4 up to F4 and back down to C4. C3 is 262 Hz; F3 is 349 Hz.

Assuming that the player moves the slide at a constant speed, how does frequency vary with time? Is a trombone glissando more like a linear or exponential chirp?

Here's the first part of the wave:

In [ ]:
low = 262
high = 340
signal = TromboneGliss(high, low)
wave1 = signal.make_wave(duration=1)
wave1.apodize()
wave1.make_audio()

And the second part:

In [ ]:
signal = TromboneGliss(low, high)
wave2 = signal.make_wave(duration=1)
wave2.apodize()
wave2.make_audio()

Putting them together:

In [ ]:
wave = wave1 | wave2
wave.make_audio()

Here's the spectrogram:

In [ ]:
sp = wave.make_spectrogram(1024)
sp.plot(high=40)

**Exercise 3.4**  Find or make a recording of a glissando and plot a spectrogram of the
first few seconds.  

In [ ]:
wave = thinkdsp.read_wave('72475__rockwehrmann__glissup02.wav')
wave.make_audio()

In [ ]:
wave.make_spectrogram(512).plot(high=50)

**Exercise 3.5**  Make or find a recording of a series of vowel sounds and look at the
spectrogram.  Can you identify different vowels?

In [ ]:
wave = thinkdsp.read_wave('87778__marcgascon7__vocals.wav')
wave.make_audio()

In [ ]:
wave.make_spectrogram(512).plot(high=25)